**CLASIFICACIÓN PARTIENDO DE IMÁGENES Y DATOS NUMÉRICOS Y CATEGÓRICOS**

En este notebook vamos a tratar de clasificar los apartamentos según su precio en baratos, medios o caros. Para ello estableceremos los límites en 50€ para los baratos y 150€ para los caros.
Dicha predicción se va a hacer a partir de dos fuentes de datos distintas: por un lado las imágenes que tenemos en el dataset de airbnb que venimos usando en las prácticas de este Bootcamp y por otro los datos numéricos y categóricos de dicho dataset.

En primer lugar nos descargamos el fichero de internet y lo copiamos en un directorio local de My Drive donde tenemos recogido todo el entorno de esta práctica.
A continuación hacemos lo mismo con las imágenes de cada una de las entradas. Usamos la vista en miniatura que sacamos de la URL de dicho fichero.

También montamos el google collab con My Drive para tenerlo vinculado.

Estos pasos solo hay que realizarlos la primera vez, una vez que tenemos los ficheros en My Drive se pueden saltar y pasamos a cargar los datos directamente desde dicho directorio.

In [ ]:
# nos descargamos el dataset de OpenDataSoft
!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

!ls -lah

--2020-06-25 06:01:03--  https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.249.199.226, 34.248.20.69
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘airbnb-listings.csv’

airbnb-listings.csv     [  <=>               ]  54.19M  2.77MB/s    in 50s     

2020-06-25 06:02:01 (1.09 MB/s) - ‘airbnb-listings.csv’ saved [56826824]

total 55M
drwxr-xr-x 1 root root 4.0K Jun 25 06:01 .
drwxr-xr-x 1 root root 4.0K Jun 25 05:59 ..
-rw-r--r-- 1 root root  55M Jun 25 06:02 airbnb-listings.csv
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwx------ 4 root root 4.0K Jun 

In [ ]:
!ls -lah

total 16K
drwxr-xr-x 1 root root 4.0K Jun 17 16:18 .
drwxr-xr-x 1 root root 4.0K Jun 26 05:15 ..
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwxr-xr-x 1 root root 4.0K Jun 17 16:18 sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!cp airbnb-listings.csv "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"

In [ ]:
# aquí creamos nuestra estructura de datos, que va a consistir en la url de la
# imagen y un índice para saber donde insertarla en nuestro array
images_paths = [[i, img_url] for i, img_url in enumerate(full_df['Thumbnail Url'])]
images_paths[:5]

[[0,
  'https://a0.muscache.com/im/pictures/cffe393a-0d84-4fd5-ab4c-a62e067c1b0d.jpg?aki_policy=small'],
 [1,
  'https://a0.muscache.com/im/pictures/ea919e56-aa99-4d5d-a129-1edf0d117d6a.jpg?aki_policy=small'],
 [2,
  'https://a0.muscache.com/im/pictures/57011236/eea5c213_original.jpg?aki_policy=small'],
 [3,
  'https://a0.muscache.com/im/pictures/974f0245-55c2-4e8c-b9bf-14c1c975c798.jpg?aki_policy=small'],
 [4,
  'https://a0.muscache.com/im/pictures/c2dde263-20dd-43af-8c6b-be636c2c0ce1.jpg?aki_policy=small']]

In [ ]:
import imageio as io
import cv2

# esta es la función que se descargará la imagen y devolverá la imagen y el 
# índice indicando la posición donde se incrustará la imagen en nuestro array
def get_image(data_url, target_size=(224, 224)):
    idx, url = data_url
    try:
        img = io.imread(url)
        # hay alguna imagen en blanco y negro y daría error al incluirla en 
        # nuestro array de imagenes que tiene 3 canales, así que convertimos
        # todas las imágenes que tengan menos de 3 dimensiones a color
        if img.ndim < 3:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img = cv2.resize(img, dsize=target_size)
        return img, idx
    except IOError as err:
        return (None, idx)

In [ ]:
import numpy as np

# en este array iremos incrustando las imágenes conforme las vayamos obteniendo
loaded_images = np.zeros((len(images_paths), 224, 224, 3), dtype=np.uint8)

# y en este array llevaremos un control de cuales se han cargado correctamente
# y cuales no
was_loaded = np.zeros(len(images_paths))

In [ ]:
import concurrent
from tqdm import tqdm

# creamos un pool de procesos que se irán descargando las imágenes
# por defecto, se crearán tantos como CPUs tenga vuestra máquina
with concurrent.futures.ProcessPoolExecutor() as executor:
    # procesamos la lista de urls de imágenes paralelizandola con el pool de procesos
    for (img, idx) in tqdm(executor.map(get_image, images_paths), total=len(images_paths)):
        # metemos la imagen en nuestro array
        if img is not None:
            loaded_images[idx] = img
            was_loaded[idx] = 1
        else:
            was_loaded[idx] = 0

print('Terminado!')
print(f'Total de imágenes recuperadas correctamente: {sum(was_loaded)}/{len(images_paths)}')

100%|██████████| 14001/14001 [08:14<00:00, 28.29it/s]

Terminado!
Total de imágenes recuperadas correctamente: 11271.0/14001


In [ ]:
# guardamos las imágenes (y yo os recomiendo que os lo guardéis en GDrive para evitar tener que repetir esto)
np.save('images.npy', loaded_images)
np.save('was_loaded.npy', was_loaded)

In [ ]:
# almacenamos las imagenes en nuestro drive
!cp images.npy "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"
!cp was_loaded.npy "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"

Esta parte de descarga, montado y copiado solo hace falta ejecutarla la primera vez. Una vez que lo tenemos copiado solo necesitamos cargarlo directamente.

A partir de aquí empieza nuestro ejercicio de clasificación.

Como hábito de buena costumbre, para no incurrir en errores involuntarios, en primer lugar se va a dividir el dataset original en train, validation y test.

Se trabaja únicamente con el de train con el objetivo de elegir un modelo. Eso se verifica con el conjunto de validation y finalmente se aplica ese "entrenamiento" al bloque de test.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


#hacemos la divisón en train, val y test
full_df = pd.read_csv('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/airbnb-listings.csv', sep=';', decimal='.')
full_train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)
train, val = train_test_split(full_train, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de validación: {val.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=';', decimal='.', index=True)
val.to_csv('./val.csv', sep=';', decimal='.', index=True)
test.to_csv('./test.csv', sep=';', decimal='.', index=True)


Dimensiones del dataset de training: (8960, 89)
Dimensiones del dataset de validación: (2240, 89)
Dimensiones del dataset de test: (2801, 89)


In [ ]:
#cargamos las imágenes desde el directorio de My Drive (ya las habíamos descargado previamente)
images  = np.load('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/images.npy')
was_loaded  = np.load('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/was_loaded.npy')

#cargamos los datos ya divididos en train, val y test
df_train = pd.read_csv('./train.csv', sep=';', decimal='.')
df_val = pd.read_csv('./val.csv', sep=';', decimal='.')
df_test = pd.read_csv('./test.csv', sep=';', decimal='.')

#usando el índice de la división anterior obtenemos los conjuntos de test, val y test en las imágenes
train_imgs = images[df_train['Unnamed: 0']]
val_imgs = images[df_val['Unnamed: 0']]
test_imgs = images[df_test['Unnamed: 0']]

train_was_loaded = was_loaded[df_train['Unnamed: 0']]
val_was_loaded = was_loaded[df_val['Unnamed: 0']]
test_was_loaded = was_loaded[df_test['Unnamed: 0']]

print(f'Dimensiones del dataset de training: {train_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_imgs.shape}')

print(f'Dimensiones del dataset de training: {train_was_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_was_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_was_loaded.shape}')


Dimensiones del dataset de training: (8960, 224, 224, 3)
Dimensiones del dataset de validación: (2240, 224, 224, 3)
Dimensiones del dataset de test: (2801, 224, 224, 3)
Dimensiones del dataset de training: (8960,)
Dimensiones del dataset de validación: (2240,)
Dimensiones del dataset de test: (2801,)


In [ ]:
# nos quedamos con los datos e imágenes para los que hemos podido encontrar imágenes
train_imgs_loaded = train_imgs[train_was_loaded == 1]
val_imgs_loaded = val_imgs[val_was_loaded == 1]
test_imgs_loaded = test_imgs[test_was_loaded == 1]

train_with_imgs = df_train[train_was_loaded == 1]
val_with_imgs = df_val[val_was_loaded == 1]
test_with_imgs = df_test[test_was_loaded == 1]

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_imgs_loaded.shape}')

print(f'Dimensiones del dataset de training: {train_with_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_with_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_with_imgs.shape}')

Dimensiones del dataset de training: (7204, 224, 224, 3)
Dimensiones del dataset de validación: (1790, 224, 224, 3)
Dimensiones del dataset de test: (2277, 224, 224, 3)
Dimensiones del dataset de training: (7204, 90)
Dimensiones del dataset de validación: (1790, 90)
Dimensiones del dataset de test: (2277, 90)


In [ ]:
#definimos la función de procesado de datos categóricos y numéricos
def preprocesado(train, val, test):
  
  #eliminamos las columnas que no aportan
  train.drop(['ID','Scrape ID','Last Scraped','Host ID','Calendar last Scraped','Listing Url','Thumbnail Url',
         'Medium Url','Picture Url','XL Picture Url','Host URL','Host Thumbnail Url','Host Picture Url',
        'Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access',
         'Interaction','House Rules','Host Name','Host About','Street','Host Location','State','Market',
         'Smart Location','Country Code','Country','Geolocation','Weekly Price','Monthly Price',
         'Host Acceptance Rate','Experiences Offered','Has Availability','License','Jurisdiction Names','Square Feet','City'], 
        axis=1, inplace=True)
  val.drop(['ID','Scrape ID','Last Scraped','Host ID','Calendar last Scraped','Listing Url','Thumbnail Url',
         'Medium Url','Picture Url','XL Picture Url','Host URL','Host Thumbnail Url','Host Picture Url',
        'Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access',
         'Interaction','House Rules','Host Name','Host About','Street','Host Location','State','Market',
         'Smart Location','Country Code','Country','Geolocation','Weekly Price','Monthly Price',
         'Host Acceptance Rate','Experiences Offered','Has Availability','License','Jurisdiction Names','Square Feet','City'], 
        axis=1, inplace=True)
  test.drop(['ID','Scrape ID','Last Scraped','Host ID','Calendar last Scraped','Listing Url','Thumbnail Url',
         'Medium Url','Picture Url','XL Picture Url','Host URL','Host Thumbnail Url','Host Picture Url',
        'Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access',
         'Interaction','House Rules','Host Name','Host About','Street','Host Location','State','Market',
         'Smart Location','Country Code','Country','Geolocation','Weekly Price','Monthly Price',
         'Host Acceptance Rate','Experiences Offered','Has Availability','License','Jurisdiction Names','Square Feet','City'], 
        axis=1, inplace=True)
  
  #PRICE
  #imputamos valores vacíos con la media de train
  MeanPriceTrain = train['Price'].mean()
  train['Price'].fillna(MeanPriceTrain, inplace=True)
  val['Price'].fillna(MeanPriceTrain, inplace=True)
  test['Price'].fillna(MeanPriceTrain, inplace=True)
  #transformamos variable Price a gausiana
  train['Price'] = train['Price'].apply(lambda x: np.log10(x))
  val['Price'] = val['Price'].apply(lambda x: np.log10(x))
  test['Price'] = test['Price'].apply(lambda x: np.log10(x))
  #categorizamos la variable precio en 3 tipos: barato (0), medio (1) y caro (2).
  train['Cat_Price'] = train['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
  val['Cat_Price'] = val['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
  test['Cat_Price'] = test['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
  
  
  #FECHAS
  train['Host Since'] = pd.to_datetime(train['Host Since'], format="%Y-%m-%d")
  train['First Review'] = pd.to_datetime(train['First Review'], format="%Y-%m-%d")
  train['Last Review'] = pd.to_datetime(train['Last Review'], format="%Y-%m-%d")
  train['Host Since'] = train['Host Since'].apply(lambda x: 2017 - x.year)
  train['First Review'] = train['First Review'].apply(lambda x: 2017 - x.year)
  train['Last Review'] = train['Last Review'].apply(lambda x: 2017 - x.year)

  val['Host Since'] = pd.to_datetime(val['Host Since'], format="%Y-%m-%d")
  val['First Review'] = pd.to_datetime(val['First Review'], format="%Y-%m-%d")
  val['Last Review'] = pd.to_datetime(val['Last Review'], format="%Y-%m-%d")
  val['Host Since'] = val['Host Since'].apply(lambda x: 2017 - x.year)
  val['First Review'] = val['First Review'].apply(lambda x: 2017 - x.year)
  val['Last Review'] = val['Last Review'].apply(lambda x: 2017 - x.year)

  test['Host Since'] = pd.to_datetime(test['Host Since'], format="%Y-%m-%d")
  test['First Review'] = pd.to_datetime(test['First Review'], format="%Y-%m-%d")
  test['Last Review'] = pd.to_datetime(test['Last Review'], format="%Y-%m-%d")
  test['Host Since'] = test['Host Since'].apply(lambda x: 2017 - x.year)
  test['First Review'] = test['First Review'].apply(lambda x: 2017 - x.year)
  test['Last Review'] = test['Last Review'].apply(lambda x: 2017 - x.year)

  #Imputamos valores en variables categóricas donde tomamos la moda para los valores que faltan.
  #Lo extraemos en una variable disinta para cada columna con la intención de aplicar el mismo valor en val y test
  ModeHSTrain = train['Host Since'].mode()[0]
  ModeHLCTrain = train['Host Listings Count'].mode()[0]
  ModeHTLCTrain = train['Host Total Listings Count'].mode()[0]
  ModeBathroomsTrain = train['Bathrooms'].mode()[0]
  ModeBedroomsTrain = train['Bedrooms'].mode()[0]
  ModeBedsTrain = train['Beds'].mode()[0]

  train['Host Since'].fillna(ModeHSTrain, inplace=True)
  train['Host Listings Count'].fillna(ModeHLCTrain, inplace=True)
  train['Host Total Listings Count'].fillna(ModeHTLCTrain, inplace=True)
  train['Bathrooms'].fillna(ModeBathroomsTrain, inplace=True)
  train['Bedrooms'].fillna(ModeBedroomsTrain, inplace=True)
  train['Beds'].fillna(ModeBedsTrain, inplace=True)

  val['Host Since'].fillna(ModeHSTrain, inplace=True)
  val['Host Listings Count'].fillna(ModeHLCTrain, inplace=True)
  val['Host Total Listings Count'].fillna(ModeHTLCTrain, inplace=True)
  val['Bathrooms'].fillna(ModeBathroomsTrain, inplace=True)
  val['Bedrooms'].fillna(ModeBedroomsTrain, inplace=True)
  val['Beds'].fillna(ModeBedsTrain, inplace=True)
  test['Host Since'].fillna(ModeHSTrain, inplace=True)
  test['Host Listings Count'].fillna(ModeHLCTrain, inplace=True)
  test['Host Total Listings Count'].fillna(ModeHTLCTrain, inplace=True)
  test['Bathrooms'].fillna(ModeBathroomsTrain, inplace=True)
  test['Bedrooms'].fillna(ModeBedroomsTrain, inplace=True)
  test['Beds'].fillna(ModeBedsTrain, inplace=True)

  #Imputamos valores en variables lineales donde tomamos la media para los valores que faltan
  #Lo extraemos en una variable disinta para cada columna con la intención de aplicar el mismo valor en val y test
  MeanRSRatingTrain = train['Review Scores Rating'].mean()
  MeanRSAccuracyTrain = train['Review Scores Accuracy'].mean()
  MeanRSCleanlinessTrain = train['Review Scores Cleanliness'].mean()
  MeanRSCheckinTrain = train['Review Scores Checkin'].mean()
  MeanRSCommunicationTrain = train['Review Scores Communication'].mean()
  MeanRSLocationTrain = train['Review Scores Location'].mean()
  MeanRSValueTrain = train['Review Scores Value'].mean()

  train['Review Scores Rating'].fillna(MeanRSRatingTrain, inplace=True)
  train['Review Scores Accuracy'].fillna(MeanRSAccuracyTrain, inplace=True)
  train['Review Scores Cleanliness'].fillna(MeanRSCleanlinessTrain, inplace=True)
  train['Review Scores Checkin'].fillna(MeanRSCheckinTrain, inplace=True)
  train['Review Scores Communication'].fillna(MeanRSCommunicationTrain, inplace=True)
  train['Review Scores Location'].fillna(MeanRSLocationTrain, inplace=True)
  train['Review Scores Value'].fillna(MeanRSValueTrain, inplace=True)
  val['Review Scores Rating'].fillna(MeanRSRatingTrain, inplace=True)
  val['Review Scores Accuracy'].fillna(MeanRSAccuracyTrain, inplace=True)
  val['Review Scores Cleanliness'].fillna(MeanRSCleanlinessTrain, inplace=True)
  val['Review Scores Checkin'].fillna(MeanRSCheckinTrain, inplace=True)
  val['Review Scores Communication'].fillna(MeanRSCommunicationTrain, inplace=True)
  val['Review Scores Location'].fillna(MeanRSLocationTrain, inplace=True)
  val['Review Scores Value'].fillna(MeanRSValueTrain, inplace=True)
  test['Review Scores Rating'].fillna(MeanRSRatingTrain, inplace=True)
  test['Review Scores Accuracy'].fillna(MeanRSAccuracyTrain, inplace=True)
  test['Review Scores Cleanliness'].fillna(MeanRSCleanlinessTrain, inplace=True)
  test['Review Scores Checkin'].fillna(MeanRSCheckinTrain, inplace=True)
  test['Review Scores Communication'].fillna(MeanRSCommunicationTrain, inplace=True)
  test['Review Scores Location'].fillna(MeanRSLocationTrain, inplace=True)
  test['Review Scores Value'].fillna(MeanRSValueTrain, inplace=True)

  #los vacíos los consideramos como desconocidos
  train['Host Neighbourhood'].fillna('Unknown', inplace=True)
  train['Host Verifications'].fillna('Unknown', inplace=True)
  train['Neighbourhood'].fillna('Unknown', inplace=True)
  train['Zipcode'].fillna('Unknown', inplace=True)
  train['Amenities'].fillna('Unknown', inplace=True)
  train['First Review'].fillna('Unknown', inplace=True)
  train['Last Review'].fillna('Unknown', inplace=True)
  val['Host Neighbourhood'].fillna('Unknown', inplace=True)
  val['Host Verifications'].fillna('Unknown', inplace=True)
  val['Neighbourhood'].fillna('Unknown', inplace=True)
  val['Zipcode'].fillna('Unknown', inplace=True)
  val['Amenities'].fillna('Unknown', inplace=True)
  val['First Review'].fillna('Unknown', inplace=True)
  val['Last Review'].fillna('Unknown', inplace=True)
  test['Host Neighbourhood'].fillna('Unknown', inplace=True)
  test['Host Verifications'].fillna('Unknown', inplace=True)
  test['Neighbourhood'].fillna('Unknown', inplace=True)
  test['Zipcode'].fillna('Unknown', inplace=True)
  test['Amenities'].fillna('Unknown', inplace=True)
  test['First Review'].fillna('Unknown', inplace=True)
  test['Last Review'].fillna('Unknown', inplace=True)

  #consideramos que donde falta un valor es porque no existe, es decir, no hay respuesta o la tasa es 0€
  train['Host Response Time'].fillna('No response', inplace=True)
  train['Host Response Rate'].fillna(0, inplace=True)
  train['Security Deposit'].fillna(0, inplace=True)
  train['Cleaning Fee'].fillna(0, inplace=True)
  train['Reviews per Month'].fillna(0, inplace=True)
  val['Host Response Time'].fillna('No response', inplace=True)
  val['Host Response Rate'].fillna(0, inplace=True)
  val['Security Deposit'].fillna(0, inplace=True)
  val['Cleaning Fee'].fillna(0, inplace=True)
  val['Reviews per Month'].fillna(0, inplace=True)
  test['Host Response Time'].fillna('No response', inplace=True)
  test['Host Response Rate'].fillna(0, inplace=True)
  test['Security Deposit'].fillna(0, inplace=True)
  test['Cleaning Fee'].fillna(0, inplace=True)
  test['Reviews per Month'].fillna(0, inplace=True)

  #transformaciones contando palabras. es algo muy sencillo, queda pendiente mejorarlo con técnicas NLP en el futuro
  train['Amenities'] = train['Amenities'].apply(lambda x: len(str(x).split(',')))
  train['Host Verifications'] = train['Host Verifications'].apply(lambda x: len(str(x).split(',')))
  train['Features'] = train['Features'].apply(lambda x: len(str(x).split(',')))
  val['Amenities'] = val['Amenities'].apply(lambda x: len(str(x).split(',')))
  val['Host Verifications'] = val['Host Verifications'].apply(lambda x: len(str(x).split(',')))
  val['Features'] = val['Features'].apply(lambda x: len(str(x).split(',')))
  test['Amenities'] = test['Amenities'].apply(lambda x: len(str(x).split(',')))
  test['Host Verifications'] = test['Host Verifications'].apply(lambda x: len(str(x).split(',')))
  test['Features'] = test['Features'].apply(lambda x: len(str(x).split(',')))

  #MeanEncoder
  categorical = ['Host Response Time', 'Host Neighbourhood', 'Neighbourhood','Neighbourhood Cleansed',
               'Neighbourhood Group Cleansed','Zipcode','Property Type','Room Type','Bed Type',
               'Calendar Updated','First Review','Last Review','Cancellation Policy']
  # En train creamos un dict para usarlo después en val y test
  mean_map = {}
  for c in categorical:
      mean = train.groupby(c)['Price'].mean()
      train[c] = train[c].map(mean)    
      mean_map[c] = mean
  for c in categorical:
    val[c] = val[c].map(mean_map[c])
  for c in categorical:
    test[c] = test[c].map(mean_map[c])
 #los valores vacíos de val y test los completo con la moda de train
  for c in categorical:
    val[c].fillna(train[c].mode()[0], inplace=True)
  for c in categorical:
    test[c].fillna(train[c].mode()[0], inplace=True)

  #eliminamos la variable Price (ya está categorizada y ya se ha usado para el MeanEncoder)
  train.drop(['Price'],axis=1, inplace=True)
  val.drop(['Price'],axis=1, inplace=True)
  test.drop(['Price'],axis=1, inplace=True)
  
  #separamos las variables de entrada de la variable objetivo
  cols = train.columns.tolist()
  Xtrain = train[cols[0:-1]]
  Xval = val[cols[0:-1]]
  Xtest = test[cols[0:-1]]

  #escalamos los valores de entrada
  cs = MinMaxScaler()
  Xtrain_Scaled = cs.fit_transform(Xtrain)
  Xval_Scaled = cs.transform(Xval)
  Xtest_Scaled = cs. transform(Xtest)

  Ytrain = train[cols[-1]]
  Yval = val[cols[-1]]
  Ytest = test[cols[-1]]

  return (Xtrain_Scaled, Xval_Scaled, Xtest_Scaled, Ytrain, Yval, Ytest)
      

In [ ]:
(Xtrain, Xval, Xtest, ytrain, yval, ytest) = preprocesado(train_with_imgs, val_with_imgs, test_with_imgs)

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_imgs_loaded.shape}')

print(f'Dimensiones del dataset de training: {Xtrain.shape}')
print(f'Dimensiones del dataset de validación: {Xval.shape}')
print(f'Dimensiones del dataset de test: {Xtest.shape}')

Dimensiones del dataset de training: (7204, 224, 224, 3)
Dimensiones del dataset de validación: (1790, 224, 224, 3)
Dimensiones del dataset de test: (2277, 224, 224, 3)
Dimensiones del dataset de training: (7204, 47)
Dimensiones del dataset de validación: (1790, 47)
Dimensiones del dataset de test: (2277, 47)


In [ ]:
#Redimensionamos las imágenes de entrada. Estoy teniendo problemas de RAM y no puedo ejecutarlo
#con 224x224 no puedo escalar /255. Con 112x112 no puedo ejecutar el modelo
#es necesario asumir esta pérdida de información
train_imgs_loaded = np.resize(train_imgs_loaded, (train_imgs_loaded.shape[0],64, 64, train_imgs_loaded.shape[3]))
val_imgs_loaded = np.resize(val_imgs_loaded, (val_imgs_loaded.shape[0],64, 64, val_imgs_loaded.shape[3]))
test_imgs_loaded = np.resize(test_imgs_loaded, (test_imgs_loaded.shape[0],64, 64, test_imgs_loaded.shape[3]))

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de training: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de training: {test_imgs_loaded.shape}')

Dimensiones del dataset de training: (7204, 64, 64, 3)
Dimensiones del dataset de training: (1790, 64, 64, 3)
Dimensiones del dataset de training: (2277, 64, 64, 3)


In [ ]:
#escalamos los datos de entrada. Lo hago en celdas separadas ya que hay algún problema de RAM
#se trata de imágenes así que no hace falta centrar, solo dividimos por el máximo. 
# nos aseguramos de hacerlo como float para no perder la info de los decimales

train_imgs_loaded = train_imgs_loaded.astype('float32') / 255.


In [ ]:
val_imgs_loaded = val_imgs_loaded.astype('float32') / 255.


In [ ]:
test_imgs_loaded = test_imgs_loaded.astype('float32') / 255.

In [ ]:
from keras.utils import to_categorical

# convertimos las etiquetas a one-hot encoding
num_classes = 3
Ytrain = to_categorical(ytrain, num_classes)
Yval = to_categorical(yval, num_classes)
Ytest = to_categorical(ytest, num_classes)

Using TensorFlow backend.


En este punto ya tenemos nuestros datos de entrada (imágenes por un lado y datos numéricos y categóricos por otro) preparados. Vamos a definir nuestros modelos de red neuronal para que traten cada uno de ese tipo de datos.

Para ello nos basamos en los notebooks anteriores y elegimos directamente el modelo que mejor prestaciones nos dio.

No obstante, eliminamos la última capa de cada uno de esos modelos. No queremos obtener el resultado final de la regresión por separado. Esas dos salidas las concatenamos, y ahora sí, las volvemos a introducir en un modelo que hace la regresión final.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, concatenate
from keras import backend as K
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.constraints import max_norm


# Creamos la rama de la red convolucional para tratar las imágenes
def miCNN(width, height, depth):
  model = Sequential()

  # Definimos una capa convolucional
  model.add(Conv2D(64, kernel_size=(5,5), input_shape=(64, 64, 3)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.09569))

  # Definimos una segunda capa convolucional
  model.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.09569))

  # Definimos una tercera capa convolucional
  model.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.09569))

  # Añadimos nuestro clasificador
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_constraint=max_norm(3.)))
  model.add(Dropout(0.09569))
  model.add(Dense(4, activation='relu'))

  return model

#creamos otra rama con una red neuronal para tratar los datos numéricos/categóricos
def miRed(dim):
  model = Sequential()
  model.add(Dense(8, input_dim=dim, activation="relu"))
  model.add(Dense(4, activation="relu"))
  
  return model

In [ ]:
#cogemos las salidas de las dos ramas por separado y las concatenamos
#esto es la entrada combinada de nuestro modelo final que da como resultado la regresión

dataBranch = miRed(Xtrain.shape[1])
imageBranch = miCNN(64,64,3)
combinedInput = concatenate([dataBranch.output, imageBranch.output])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=[dataBranch.input, imageBranch.input], outputs=x)

# Compilamos el modelo
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="categorical_crossentropy", 
              optimizer=opt,
							metrics=['accuracy'])

# Entrenamos el modelo
print("[INFO] training model...")
model.fit([Xtrain,train_imgs_loaded], Ytrain,
          batch_size=128,
					shuffle=True,
					epochs=10,
					validation_data=([Xval, val_imgs_loaded], Yval))

[INFO] training model...

Train on 7204 samples, validate on 1790 samples
Epoch 1/10
7204/7204 [==============================] - 11s 2ms/step - loss: 1.1086 - accuracy: 0.4213 - val_loss: 1.0717 - val_accuracy: 0.4950
Epoch 2/10
7204/7204 [==============================] - 4s 553us/step - loss: 1.0607 - accuracy: 0.4720 - val_loss: 1.0474 - val_accuracy: 0.4950
Epoch 3/10
7204/7204 [==============================] - 4s 555us/step - loss: 1.0381 - accuracy: 0.4629 - val_loss: 1.0259 - val_accuracy: 0.4480
Epoch 4/10
7204/7204 [==============================] - 4s 558us/step - loss: 1.0184 - accuracy: 0.4603 - val_loss: 1.0068 - val_accuracy: 0.4950
Epoch 5/10
7204/7204 [==============================] - 4s 556us/step - loss: 1.0011 - accuracy: 0.4713 - val_loss: 0.9901 - val_accuracy: 0.4950
Epoch 6/10
7204/7204 [==============================] - 4s 558us/step - loss: 0.9860 - accuracy: 0.4714 - val_loss: 0.9756 - val_accuracy: 0.4950
Epoch 7/10
7204/7204 [=============================

Por último evaluamos el modelo con el conjunto de test

In [ ]:
# Evaluamos el modelo
scores = model.evaluate([Xtest,test_imgs_loaded], Ytest)

print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])


2277/2277 [==============================] - 1s 284us/step
Loss: 0.943
Accuracy: 0.477


seguimos teniendo el problema de la clasificación al 47% o 49%. En la siguiente celda se pueden ver las predicciones

In [ ]:
pred = model.predict([Xval,val_imgs_loaded])
#imprimo  unas cuantas aleatorias, y siempre da lo mismo
pred[100:150,:]

array([[0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.41431552, 0.41137004, 0.17431442],
       [0.

**CONCLUSIÓN FINAL**

Viendo los resultados tenemos que elegir como el mejor de los 3 cláramente el modelo que trabaja con datos numéricos y categóricos. Nos dio una precisión del 85%.

En el momento que hemos introducido las imágenes la precisión ha bajado mucho, casi al 50%.

La principal razón puede ser el redimensionamiento que hemos tenido que hacer a la baja de las imágenes, ya que con el tamaño original no lo podía procesar.